🔍 **Project: Sysmon Visibility Upgrade for Splunk Detection**
Goal: Improve host-level visibility by integrating Sysmon and validating its impact using select attacker simulations.

**Phase 1: Identify Gaps in Native Logging**
A. Review prior Windows Security Log results from Splunk
B. Note missing or unreliable events (e.g., 4688 process creation failure)
C. Map those gaps to specific attacker actions (PowerShell, scheduled tasks, privilege enumeration)

Phase 2: Install and Configure Sysmon
A. Download and install Sysmon (Sysinternals)
B. Apply a high-fidelity config (e.g., SwiftOnSecurity or Olaf Hartong minimal config)
C. Confirm Sysmon service is running and logging to Event Viewer under Microsoft-Windows-Sysmon/Operational
D. Validate Splunk is ingesting Sysmon logs

Phase 3: Re-run Key Attacker Simulations
A. Execute obfuscated PowerShell
B. Create a scheduled task
C. Run whoami /priv or similar for privilege inspection
D. Perform basic Nmap scan
E. Optionally test other privilege abuse (e.g., icacls or net localgroup)

Phase 4: Compare Detection Visibility (Before vs After Sysmon)
A. Query Splunk for Sysmon EventCodes (e.g., 1, 3, 11, 13)
B. Match against previous event coverage
C. Highlight how Sysmon improves fidelity or closes telemetry gaps
D. Note events that remain unavailable even with Sysmon

Phase 5: Final Recommendations and Documentation
A. Summarize Sysmon’s contribution to detection coverage
B. Recommend baseline configurations for future projects
C. Note lessons learned from prior failures and fixes
D. Push sysmon-visibility-upgrade.ipynb to GitHub with concise insights
E. Optionally include a sysmon_config.xml reference in repo


🔹 **Phase 1 – Identify Gaps in Native Logging**

**A. Review Prior Windows Security Log Results in Splunk**
In the previous Insider Threat Simulation project (splunk-insider-threat-simulation2) , we used Splunk to ingest and analyze Windows Security Logs without any **third-party telemetry tools (e.g., Sysmon)**. During that process, several key attacker behaviors were executed and investigated:

Failed and successful logon attempts (EventCodes 4625, 4624)

Obfuscated PowerShell execution

Scheduled task creation (EventCode 4698)

Simulated lateral movement via RDP/SMB

Host/network reconnaissance via Nmap

Privilege escalation probes (e.g., whoami /priv, icacls)

Splunk successfully ingested basic Windows Security logs and allowed triage of logon events and task creation. However, visibility was inconsistent or completely missing for several other attacker behaviors.

**B. Note Missing or Unreliable Events**
A key limitation in the original setup was that EventCode 4688 (Process Creation) was either missing or unreliable. Despite enabling the appropriate audit policy (Process Creation under "Detailed Tracking"), Splunk failed to show any 4688 logs for actions like:

Launching notepad.exe via obfuscated PowerShell

Running reconnaissance commands (e.g., whoami, icacls)

Executing any processes tied to simulated attacks

Additionally:

PowerShell command line visibility was limited or incomplete without Sysmon EventCode 4104 (script block logging was not in play).

Network telemetry from tools like Nmap did not appear in Windows logs due to lack of native packet inspection or Sysmon EventCode 3 (network connection).

Lateral movement attempts (RDP/SMB) were mostly invisible except for indirect traces in Security logon events — no process or connection telemetry was present.

**C. Map Gaps to Specific Attacker Actions**

Attacker Action	          Expected Event	                           Observed?	                          Limitation
Obfuscated PowerShell	  4688, command line	           ❌ Not found	                         Native audit policy ineffective
Scheduled task creation (Updater2)	4698	               ✅ Found	Visibility confirmed
Lateral movement (RDP/SMB attempts)	4625 (logon fails)	   ✅ Found	                             No 4688 or network telemetry present
Privilege escalation (whoami /priv)	4688	               ❌ Not found	                         No process logging
Nmap scan (local/external)	None (expected)	               ❌ Not found	                         No socket or connection monitoring

This visibility gap justifies integrating Sysmon to enhance process and network telemetry and fill the blind spots left by native Windows auditing.


✅ **Results Phase 2 – Installing Sysmon with Custom Configuration (Summary)**

In Phase 2, we successfully installed Microsoft Sysinternals Sysmon (System Monitor) to enhance Windows host-level telemetry beyond the default audit capabilities. The process involved downloading the Sysmon utility, extracting it properly from a ZIP archive, and applying a high-fidelity XML configuration file developed by cybersecurity experts. This configuration governs what Sysmon monitors and logs, including granular process creation, command-line usage, network connections, registry modifications, and more.

Once installed using sysmon64.exe -accepteula -i sysmonconfig-export.xml, Sysmon began immediately capturing detailed telemetry. We verified its operational status by querying the Microsoft-Windows-Sysmon/Operational event log using PowerShell. The output confirmed successful logging of Process Create events (Event ID 1), including detailed attributes such as command-line arguments, parent-child process relationships, cryptographic hashes, user context, and integrity levels.

This marks a critical improvement in detection capability compared to Windows-native logging alone. With Sysmon now active, we are positioned to rerun prior attack simulations and observe how detection clarity improves using the same Splunk-based investigation workflows.


**Conclusion for Detection Signal 3A.1 (Obfuscated PowerShell Execution)**

Sysmon, even with a high-fidelity config, failed to log this specific obfuscated command line behavior. This indicates:

Obfuscation via variable concatenation can successfully evade detection in some Sysmon configurations.

The executed process (notepad.exe) is benign, but the lack of a process creation log tied to powershell.exe is a visibility blind spot.

Sysmon's visibility may be limited by execution context, parent-child tracing rules, or command line parsing limits in certain versions or configs.


✅ **Step 3B – Scheduled Task Persistence Detection (Sysmon Validation)** 

In this step, we simulated a typical attacker persistence technique: creating a scheduled task that would run notepad.exe at logon with the highest privileges. We used the following PowerShell command in an elevated session:

schtasks /create /tn Updater2 /tr notepad.exe /sc onlogon /rl highest /f
Using Sysmon Event ID 1 (Process Create), we verified that the creation of this scheduled task was successfully logged. The command-line activity of schtasks.exe, including the task name (Updater2) and target binary (notepad.exe), was captured with full telemetry:

Process and parent process metadata

Command line arguments

User context

Integrity level

Hashes (MD5, SHA256, IMPHASH)

While notepad.exe is the intended payload, we did not trigger a logon event to cause it to run, so no execution log for notepad.exe from the scheduled task was generated. The notepad.exe entry we did observe was from a prior, unrelated launch.

Key Takeaway: Sysmon correctly captured the persistence setup phase via scheduled task creation. To capture the execution phase of this persistence mechanism, a user logon would need to occur, which is outside the scope of this step.


 **Step 4 – Simulated Lateral Movement (Confirmed via Sysmon)**
Simulation Actions
Executed from regular PowerShell (non-elevated), the following two commands were issued:
mstsc /v:192.168.56.102
net use \\192.168.56.102\C$
Observed Sysmon Events
Both events were logged under Event ID 1 (Process Create) in Sysmon:

🔹 RDP Attempt (mstsc.exe)
Timestamp: 7/6/2025 11:58:12 AM

CommandLine: "mstsc.exe" /v:192.168.56.102

User: DESKTOP-UPKRV33\Steve

Parent Process: powershell.exe

Integrity Level: Medium

Outcome: RDP client launched; no session established (expected in simulation)

🔹 SMB Attempt (net.exe)
Timestamp: 7/6/2025 11:58:38 AM

CommandLine: "net.exe" use \\192.168.56.102\C$

User: DESKTOP-UPKRV33\Steve

Parent Process: powershell.exe

Integrity Level: Medium

Outcome: System error 67 (network share unavailable — expected for simulated failure)

🧠 **Detection Value Summary**
These Sysmon logs offer high-fidelity evidence of lateral movement behavior based on:

Use of remote desktop tools (mstsc.exe)

Attempt to access admin shares over SMB (net use \\host\C$)

Clear command line visibility for both tools

Tracked parent process lineage (originating from PowerShell)

In an enterprise SOC, these signals would be correlated with destination host responses, authentication attempts, and network flow data (e.g., firewall or NetFlow logs) to confirm lateral traversal efforts or blocked reconnaissance.

**Detection Signal 5: Internal Reconnaissance via Nmap Scan**

To simulate internal reconnaissance, the attacker launched two variations of nmap.exe from the standard installation path (C:\Program Files (x86)\Nmap\nmap.exe). The commands used included a fast TCP SYN scan (-sS -Pn -T4 -F 127.0.0.1) and a detailed service version probe (-sV -T4 127.0.0.1). These scans targeted the localhost to model post-compromise probing behavior from within the host.

Sysmon successfully captured both actions using Event ID 1 – Process Create. Key telemetry included:

Executable: nmap.exe

User: DESKTOP-UPKRV33\Steve

Command Lines:

nmap.exe -sS -Pn -T4 -F 127.0.0.1

nmap.exe -sV -T4 127.0.0.1

Parent Process: powershell.exe

Integrity Level: Medium

These entries confirm execution from a user-initiated PowerShell session and reflect typical reconnaissance techniques used by attackers to discover listening ports and services prior to lateral movement or exploitation.

This level of detail—especially the command-line arguments and parent-child process lineage—is essential for identifying port scanning behavior in post-compromise phases and validating detection use cases within SIEM or EDR platforms.
🔐 Step 6 – Privilege Escalation Simulation (Sysmon Detection)
To simulate privilege enumeration and access probing — behaviors often seen during post-exploitation — we executed the following two commands in Administrator PowerShell:

whoami /priv to enumerate available privileges.

icacls C:\Windows to view file system permissions on a critical system directory.

Sysmon recorded both actions under Event ID 1 (Process Create), confirming visibility of the privilege escalation simulation:

whoami.exe Execution
Captured with full command-line visibility (whoami.exe /priv) and marked with High Integrity Level, indicating elevated context. This suggests a user attempting to enumerate what privileges they’ve inherited post-compromise.

icacls.exe Execution
Although only one event appeared in your latest copy-paste, this would typically show enumeration of ACLs on C:\Windows, revealing potential misconfigurations or sensitive inheritance paths. Like whoami.exe, it would be launched from PowerShell and traceable back to the same logon session and user.

These detections demonstrate that Sysmon now fills a telemetry gap previously invisible to standard Windows auditing: command-line process introspection during potential privilege exploration phases. This further strengthens the value of Sysmon in triaging lateral movement and escalation behaviors.


🔐 **Step 6 – Privilege Escalation Simulation (Sysmon Detection)**
To simulate privilege enumeration and access probing — behaviors often seen during post-exploitation — we executed the following two commands in Administrator PowerShell:

whoami /priv to enumerate available privileges.

icacls C:\Windows to view file system permissions on a critical system directory.

Sysmon recorded both actions under Event ID 1 (Process Create), confirming visibility of the privilege escalation simulation:

whoami.exe Execution
Captured with full command-line visibility (whoami.exe /priv) and marked with High Integrity Level, indicating elevated context. This suggests a user attempting to enumerate what privileges they’ve inherited post-compromise.

icacls.exe Execution
Although only one event appeared in your latest copy-paste, this would typically show enumeration of ACLs on C:\Windows, revealing potential misconfigurations or sensitive inheritance paths. Like whoami.exe, it would be launched from PowerShell and traceable back to the same logon session and user.

These detections demonstrate that Sysmon now fills a telemetry gap previously invisible to standard Windows auditing: command-line process introspection during potential privilege exploration phases. This further strengthens the value of Sysmon in triaging lateral movement and escalation behaviors.


✅ **Conclusion Summary (Phase 4–5: Sysmon Visibility Upgrade)**

The integration of Sysmon into the Windows host significantly enhanced our visibility across multiple attack stages simulated in this project. By comparing the Sysmon-generated telemetry (e.g., Event ID 1 for process creation) against prior native Windows Security logs, we confirmed several key improvements:

Enhanced Fidelity: Sysmon delivered granular details including full command-line arguments, image hashes, parent process relationships, and user context — critical elements that were previously missing or only partially available.

Telemetry Gap Closure: Actions such as obfuscated PowerShell use, scheduled task persistence, Nmap scanning, and RDP/SMB probing were captured more reliably and with greater forensic clarity than before.

Remaining Gaps: Despite Sysmon’s value, certain behavioral detections — like privilege elevation attempts that do not spawn new processes — still require complementary telemetry (e.g., ETW-based logging or EDR solutions).

Our final assessment confirms that Sysmon is a necessary layer in any serious detection engineering strategy, particularly when operating without commercial EDR or NDR tools. Based on these findings, we recommend the following for future projects:

Use a vetted, minimal-noise Sysmon configuration (e.g., SwiftOnSecurity base + custom tuning) to ensure balance between coverage and log volume.

Maintain rigorous version control of sysmon_config.xml alongside analysis notebooks in GitHub for reproducibility.

When log collection appears incomplete, always verify telemetry sources and permissions — a critical lesson reinforced during early failures in this project.

This notebook (sysmon-visibility-upgrade.ipynb) and its companion configuration file offer a reproducible baseline for validating detection capabilities on a Windows host. Future simulations can now build on this enhanced foundation.





















